<a href="https://colab.research.google.com/github/owenant/PlayTraces/blob/main/DominionPlayTraceClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#This notebook calculates silhouette averages and clusters for Card Count and NGram playtraces for Dominion produced by the
#TableTop Games framework

In [2]:
!pip install scikit-learn-extra

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import pdb
import re
import nltk
import math
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from itertools import product, permutations, combinations
from nltk import ngrams
from nltk.probability import FreqDist
nltk.download('punkt')
from collections import Counter
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn_extra.cluster import KMedoids
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy.spatial.distance import jensenshannon
from google.colab import drive
import csv
import os
import warnings

drive.mount('/content/gdrive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/gdrive


In [4]:
#list of clustering methods to use
clustering_methods = ['KMeans', 'KMedoids', 'DBSCAN', 'SPCluster_AM','SPCluster_KNN']

#filenames and directory locations
google_drive_parent_dir = "gdrive/My Drive/Colab Notebooks/DominionPlayTraceClustering/"
card_count_data_dir = google_drive_parent_dir + "DataCardCount/"
ngram_data_dir = google_drive_parent_dir + "DataNGrams/"
card_count_data_filename = card_count_data_dir + "trace_logfile_BMWG_vs_DW_GPM100.txt"
ngram_data_filename = ngram_data_dir + "ActionsReduced_BMWG_vs_DW_GPM100.csv"
tag_for_dir_and_filenames = 'BMWG_vs_DW_GPM100'
#card_count_data_filename = card_count_data_dir + "trace_logfile_Budget_500_vs_Budget_500_GPM100_SD_NoSelfPlay.txt"
#ngram_data_filename = ngram_data_dir + "ActionsReduced_Budget500_vs_Budget500_GPM100_SD.csv"
#tag_for_dir_and_filenames = 'MCTS_b500_vs_b500_GPM100_SD'

#create new directory for output files
for method in clustering_methods:
  new_dir_path = google_drive_parent_dir + 'Results_' + method + '/' + tag_for_dir_and_filenames + '/'
  os.makedirs(new_dir_path, exist_ok=True)

  # Verify that the directory has been created
  if os.path.exists(new_dir_path):
      print(f"Directory '{new_dir_path}' created successfully.")
  else:
      print(f"Failed to create directory '{new_dir_path}'.")

#also create directory for round and score distributions
new_dir_path = google_drive_parent_dir + 'RoundAndScoreDistributions/' + tag_for_dir_and_filenames + '/'
os.makedirs(new_dir_path, exist_ok=True)

# Verify that the directory has been created
if os.path.exists(new_dir_path):
    print(f"Directory '{new_dir_path}' created successfully.")
else:
    print(f"Failed to create directory '{new_dir_path}'.")

Directory 'gdrive/My Drive/Colab Notebooks/DominionPlayTraceClustering/Results_KMeans/BMWG_vs_DW_GPM100/' created successfully.
Directory 'gdrive/My Drive/Colab Notebooks/DominionPlayTraceClustering/Results_KMedoids/BMWG_vs_DW_GPM100/' created successfully.
Directory 'gdrive/My Drive/Colab Notebooks/DominionPlayTraceClustering/Results_DBSCAN/BMWG_vs_DW_GPM100/' created successfully.
Directory 'gdrive/My Drive/Colab Notebooks/DominionPlayTraceClustering/Results_SPCluster_AM/BMWG_vs_DW_GPM100/' created successfully.
Directory 'gdrive/My Drive/Colab Notebooks/DominionPlayTraceClustering/Results_SPCluster_KNN/BMWG_vs_DW_GPM100/' created successfully.
Directory 'gdrive/My Drive/Colab Notebooks/DominionPlayTraceClustering/RoundAndScoreDistributions/BMWG_vs_DW_GPM100/' created successfully.


In [5]:
#parameters for notebook execution

#kingdom card set
kingdom_set = 'SD'

#parameters if using TAG input data
logs_from_tag = True
agent_names = ['BMWG', 'DW']
games_per_matchup = 100
no_self_play = True

#parameters for grid search for clustering methods

#number of clusters to check across all clustering methods (excluding DBSCAN)
clusters_min = 2
clusters_max = 5
clusters_stepsize = 1

#DBSCAN
minpts_min = 5
minpts_max = 50
minpts_stepsize = 5
epsilon_min = 0.1
epsilon_max = 1
epsilon_stepsize = 0.1

#Spectral clustering K-Nearest Neighbours
nearest_neighbours_min = 5
nearest_neighbours_max = 50
nearest_neighbours_stepsize = 5

#Spectral clustering radial basis function
gamma_min = 0.1
gamma_max = 1
gamma_stepsize = 0.1

#number of N-gram types to search over
ngram_min = 1
ngram_max = 2
ngram_stepsize = 1

#values of k for l_k norm
k_norms = [0.1, 0.5, 1, 2]

#threshold values for plotting probability distributions for N-Gram playtraces
thresholds = {n: 0.01 for n in range(ngram_min, ngram_max + 1, ngram_stepsize)}

In [6]:
#kingdom card types
card_types_SD = ['ARTISAN', 'BANDIT', 'BUREAUCRAT', 'CHAPEL', 'FESTIVAL', 'GARDENS', 'SENTRY', 'THRONE_ROOM', 'WITCH',
                 'WORKSHOP', 'CURSE', 'PROVINCE', 'DUCHY', 'ESTATE', 'GOLD', 'SILVER', 'COPPER']
card_types_FG1E = ['CELLAR','MARKET','MILITIA','MINE','MOAT','REMODEL','SMITHY','VILLAGE',
                'WOODCUTTER','WORKSHOP','CURSE','PROVINCE', 'DUCHY', 'ESTATE', 'GOLD', 'SILVER', 'COPPER']

if kingdom_set == 'SD':
  card_types = card_types_SD
elif kingdom_set == 'FG1E':
  card_types = card_types_FG1E
else:
  print('Unrecognised kingdom card set')

In [7]:
#functions to process data from TAG

def gameID_to_matchup(game_id, player_no, matchup_list, no_games_per_matchup, min_game_id):
    game_group = int((game_id - min_game_id)/no_games_per_matchup)
    matchup = matchup_list[game_group]
    agent1, agent2 = matchup
    if player_no == 0:
        return agent1
    else:
        return agent2

def add_TAG_agent_names(agent_names, games_per_match_up, no_self_play, logs_from_tag, data):
  NoOfGames = len(data['GameID'].unique())
  min_GameID = data['GameID'].min()

  #first generate match-ups
  matchups = []
  if no_self_play:
    matchups = list(permutations(agent_names, 2))
  else:
      for agent1 in agent_names:
          for agent2 in agent_names:
              matchups.append((agent1, agent2))

  #add agent names to data set
  data['AgentName'] = data.apply(lambda row: gameID_to_matchup(row['GameID'], row['Player'], matchups, games_per_matchup, min_GameID), axis = 1)

  #finally we also add the name of the agent of the opponent
  min_GameID = data['GameID'].min()
  data['Opponent'] = data.apply(lambda row: 1.0 if row['Player'] == 0.0 else 0.0, axis = 1)
  if logs_from_tag:
      data['AgentNameOpponent'] = data.apply(lambda row: gameID_to_matchup(row['GameID'], row['Opponent'], matchups, games_per_matchup, min_GameID), axis = 1)
  else:
      gameid_to_players_dict = data.groupby('GameID')['AgentName'].apply(list).to_dict()
      data['AgentNameOpponent'] = data.apply(lambda row: other_dict_element(gameid_to_players_dict, row['GameID'], row['AgentName']), axis = 1)


#functions to process card count data
def copy_final_deck_at_game_end(group, roundMax, noPlayers):
  #This function repeatedly copies the final decks of two players at the game end, so that the game is extended to
  #have roundMax rounds
  final_round = int(group['Round'].max())
  if (roundMax-1) == final_round:
      #in this case we dont need to extend the play trace
      return group
  else:
      final_row_copy = pd.concat([group.iloc[-noPlayers:]] * ((roundMax-1) - final_round), ignore_index=True)
      #we need to update the Round counter so that every other row it increments by one
      final_row_copy['Round'] = [final_round + 1 + i // 2 for i in range(((roundMax-1) - final_round)*2)]
      return pd.concat([group, final_row_copy], ignore_index=True)

#given a dictionary whose elements are lists of length two, grab the other element not given by elem
def other_dict_element(my_dict, my_key, my_elem):
    index_of_given_element = my_dict[my_key].index(my_elem)
    index_of_other_element =  1 if (index_of_given_element == 0) else 0
    return my_dict[my_key][index_of_other_element]

def process_card_count_data(cardcount_filename, agent_names, games_per_matchup,
                            no_self_play, card_types, logs_from_tag):
  data  = pd.read_csv(cardcount_filename, sep = '\t')
  add_TAG_agent_names(agent_names, games_per_matchup, no_self_play, logs_from_tag, data)
  index_cols = ['Player', 'GameID']
  non_card_types_round_indep_cols = ['AgentName', 'AgentNameOpponent', 'Win', 'FinalScore', 'TotalRounds']
  cols = index_cols + non_card_types_round_indep_cols + ['Round'] + card_types #final set of cols to keep
  data = data[data['Turn'] == 1] #only want cards at end of round
  data = data.loc[:, cols]

  #freeze decks and copy to max round number
  no_players = 2
  gameLengths = data.groupby(['GameID'])['Round'].max()
  maxNoOfRounds = int(gameLengths.max()) + 1 #round counter starts at zero
  noOfGames = len(data['GameID'].unique())
  data = data.groupby('GameID').apply(copy_final_deck_at_game_end, maxNoOfRounds, no_players).reset_index(drop = True)

  #check shape of data
  print("Card count shape check:")
  print("Expected no rows: " + str(maxNoOfRounds*no_players*noOfGames))
  print("Expected no of cols: " + str(len(card_types)+8))
  print(data.shape)

  return data

def flatten_card_count_data(cardcount_data, card_types):
  #next we need to flatten our data so that each trace is a single row.
  #We also drop the round label as it is redundant
  #and it will get reintroduced when flattening through the revised column names

  #first create dataframe consisting of only non card type data types that are round
  #independent
  index_cols = ['Player', 'GameID']
  non_card_types_round_indep_cols = ['AgentName', 'AgentNameOpponent', 'Win', 'FinalScore', 'TotalRounds']
  non_card_data_round_indep = cardcount_data[index_cols + non_card_types_round_indep_cols].drop_duplicates()

  #next need to Group by Player and GameID and then flatten card data by round
  traces_tmp = cardcount_data[index_cols + card_types]
  gameLengths = cardcount_data.groupby(['GameID'])['Round'].max()
  maxNoOfRounds = int(gameLengths.max()) + 1 #round counter starts at zero
  cols = [card_types[i] + "_R" + str(r)
          for r in range(0, maxNoOfRounds) for i in range(0, len(card_types))]

  extended_traces_flat = traces_tmp.groupby(index_cols).apply(lambda df: df[card_types].values.flatten())
  extended_traces_flat = pd.DataFrame(extended_traces_flat, columns = ['Trace']).reset_index()
  extended_traces_flat = pd.concat([extended_traces_flat[index_cols], extended_traces_flat['Trace'].apply(pd.Series)], axis=1)
  extended_traces_flat.columns = index_cols + cols

  #next we add back in the round independent data
  extended_traces_flat = pd.merge(non_card_data_round_indep, extended_traces_flat, on = index_cols)

  return extended_traces_flat

#functions to process NGram data
def format_action(action, cardtypes):
  #there are various types of actions we need to format to identify these we use
  #regular expressions
  pattern_list = []
  pattern_list.append(re.compile(r'End Current Phase'))
  pattern_list.append(re.compile(r'BuyCard: (' + '|'.join(cardtypes) + r') by player (0|1)'))
  pattern_list.append(re.compile(r'(' + '|'.join(cardtypes) + r') : Player (0|1)'))
  pattern_list.append(re.compile(r'GainCard: (' + '|'.join(cardtypes) + r') by player (0|1)'))
  pattern_list.append(re.compile(r'Player (0|1) trashes a (' + '|'.join(cardtypes) + r') from (?:HAND|DISCARD)'))
  pattern_list.append(re.compile(r'DoNothing'))
  pattern_list.append(re.compile(r'Player (0|1) moves (' + '|'.join(cardtypes) + r') from HAND to DRAW of player (0|1) \(visible: (?:true|false)\)'))
  pattern_list.append(re.compile(r'Reveals Hand'))
  pattern_list.append(re.compile(r'Sentry .*$')) #captures playing a sentry and then discard/trash two cards
  pattern_list.append(re.compile(r'Player (0|1) discards (' + '|'.join(cardtypes) + r')'))
  pattern_list.append(re.compile(r'Player (0|1) reveals a (' + '|'.join(cardtypes) + r')'))

  match_list = [None] * len(pattern_list)

  pattern_to_string_map = ['ECP', 'BUY', 'PLAY', 'GAIN', 'TRASHES',
                            'DONOTHING', 'MOVES', 'REVEALSHAND', 'PLAYSSENTRY',
                            'DISCARDS', 'REVEALS']

  for index in range(0, len(pattern_list)):
    matched = pattern_list[index].match(action)
    pattern_index = index
    if matched != None:
      break

  if matched == None:
    pdb.set_trace()
    raise Exception("Can't match action description")

  if pattern_index in [0, 5, 7, 8]:
    formatted_action =  pattern_to_string_map[pattern_index]
  elif pattern_index in [1, 2, 3]:
    matched_card =  matched.group(1)
    formatted_action =  pattern_to_string_map[pattern_index]  + matched_card
  else:
    matched_card = matched.group(2)
    formatted_action =  pattern_to_string_map[pattern_index]  + matched_card

  return formatted_action

def process_ngram_data(actions_filename, agent_names, games_per_match_up, no_self_play,
                       card_types, ngram_min, ngram_max, ngram_stepsize, logs_from_tag):
  data  = pd.read_csv(actions_filename)
  data = data[['GameID', 'Player', 'Round','Turn','ActionDescription']]
  add_TAG_agent_names(agent_names, games_per_match_up, no_self_play, logs_from_tag, data)
  data['ProcAction'] = data.apply(lambda row: format_action(row['ActionDescription'], card_types), axis = 1)
  data = data.groupby(['GameID', 'Player','AgentName', 'AgentNameOpponent'])['ProcAction'].agg(lambda x: ' '.join(x)).reset_index()

  for n in range(ngram_min, ngram_max +1, ngram_stepsize):
    col_name = 'NGrams_' + str(n)
    data[col_name] = data.apply(lambda row: list(ngrams(nltk.word_tokenize(row['ProcAction']),n)), axis = 1)

  return data


In [8]:
#process data
traces_cardcount = process_card_count_data(card_count_data_filename, agent_names, games_per_matchup, no_self_play, card_types, logs_from_tag)
traces_ngrams = process_ngram_data(ngram_data_filename, agent_names, games_per_matchup, no_self_play,
                       card_types, ngram_min, ngram_max, ngram_stepsize, logs_from_tag)
print(len(traces_ngrams))

Card count shape check:
Expected no rows: 12000
Expected no of cols: 25
(12000, 25)
400


In [9]:
#function to plot score and round distributions and output to file
def plot_score_round_distributions(outputfilename, card_count_data):
  fig, axs = plt.subplots(2, 1)
  grouped_data = card_count_data.groupby('GameID')
  score_data = grouped_data['FinalScore'].unique().explode()
  axs[0].hist(score_data, bins=np.arange(score_data.min(), score_data.max()+1))
  axs[0].set_xlabel('Final score')
  axs[0].set_ylabel('Number of games')
  axs[0].set_title('Score distribution')
  round_data = grouped_data['TotalRounds'].unique().explode()
  axs[1].hist(round_data, bins=np.arange(round_data.min(), round_data.max()+1))
  axs[1].set_xlabel('Number of rounds')
  axs[1].set_ylabel('Number of games')
  axs[1].set_title('Round distribution')
  fig.tight_layout()
  plt.savefig(outputfilename +'.png', format = 'png')
  plt.close()


In [10]:
#remove outliers based on thresholds for score and length of game
score_threshold = 300
round_threshold = 300
print("Round and score distribution before outliers removed:")
outputfilename = google_drive_parent_dir + 'RoundAndScoreDistributions/' + tag_for_dir_and_filenames + '/' + 'RoundAndScoreDistribution_' + tag_for_dir_and_filenames
plot_score_round_distributions(outputfilename, traces_cardcount)

traces_cardcount = traces_cardcount[(traces_cardcount['FinalScore'] <= score_threshold)
                                           & (traces_cardcount['TotalRounds'] <= round_threshold)]
new_game_id_list =  traces_cardcount['GameID'].unique()
traces_ngrams = traces_ngrams[traces_ngrams['GameID'].isin(new_game_id_list)]

Round and score distribution before outliers removed:


In [11]:
print("Round and score distribution after outliers removed:")
outputfilename = google_drive_parent_dir + 'RoundAndScoreDistributions/' + tag_for_dir_and_filenames + '/' + 'RoundAndScoreDistribution_no_outliers_' + tag_for_dir_and_filenames
plot_score_round_distributions(outputfilename, traces_cardcount)

Round and score distribution after outliers removed:


In [12]:
#flatten card count data so that we have a playtrace per row
traces_cardcount = flatten_card_count_data(traces_cardcount, card_types)
print(len(traces_cardcount))
print(len(traces_ngrams))

400
400


In [13]:
#Compute the all ngrams list by taking the total list of all observed n-grams for this tournament
all_ngrams_list = {}
for n in range(ngram_min, ngram_max + 1, ngram_stepsize):
  col_name = 'NGrams_' + str(n)
  all_ngrams_list[n] = []
  for row in traces_ngrams[col_name]:
    for gram in row:
      if gram not in all_ngrams_list[n]:
        all_ngrams_list[n].append(gram)
  print("Total number of Ngrams for N=" + str(n) + ":" + str(len(all_ngrams_list[n])))

Total number of Ngrams for N=1:9
Total number of Ngrams for N=2:29


In [14]:
#functions to support NGram analysis

#function to compute N-gram probabilities, returns either an array with probability values
#in the same order as ngrams in ngrams_all, or a dictionary with the n-grams as key
#Unobserved ngrams (i.e. ngrams in ngrams_all, that are not in the trace) are assigned
#a default probability of zero.
def calc_probabilities(ngrams_trace, ngrams_all, convertToArray = False):
    # Compute the frequency of ngrams in the trace
    frequency_counter = Counter(ngrams_trace)

    #calculate frequencies of all ngrams in ngrams_all that appear in the playtrace
    event_count = {gram: frequency_counter.get(gram, 0) for gram in ngrams_all}

    #normalise each entry with the number of n-grams observed for that trace, to convert
    #counts into probabilities
    trace_n_gram_count = sum(frequency_counter.values())
    probs = {key: value / (1.0*trace_n_gram_count) for key, value in event_count.items()}

    if convertToArray:
        probs = np.array(list(probs.values()))

    return probs

#function to take a probability dictionary and create an array
def prob_dict_to_array(prob_dict):
    return np.array(list(prob_dict.values()))

#funciton to take probability array and convert to dictionary with n-grams as keys
#assumes ordering has been maintained
def prob_array_to_dict(prob_array, ngrams_all):
    prob_dict = {}
    index = 0
    for gram in ngrams_all:
        prob_dict[gram] = prob_array[index]
        index+=1
    return prob_dict

#find the common set of ngrams between two probability dictionaries, with probabilities
#above a given threshold
def return_common_ngrams_above_threshold(prob_dict1, prob_dict2, threshold):
    common_ngrams = []
    #look for entries in the first dictionary with non-zero values
    for key, value in prob_dict1.items():
        if value > threshold:
            common_ngrams.append(key)
    #repeat for the second dictionary but avoiding duplicates
    for key, value in prob_dict2.items():
        if (value > threshold) and (key not in common_ngrams):
             common_ngrams.append(key)
    return common_ngrams

#convert a list of ngram tuples into a list of strings
def convert_ngram_tuples_to_strings(ngrams_list):
    ngrams_str = []
    for tuple_item in ngrams_list:
        tuple_str = ''
        for index, element in enumerate(tuple_item):
            if index != (len(tuple_item)-1):
                tuple_str += element + '|'
            else:
                tuple_str += element
        ngrams_str.append(tuple_str)
    return ngrams_str

In [15]:
#add columns to trace data containing arrays for probability data
for n in range(ngram_min, ngram_max + 1, ngram_stepsize):
  col_name_1 = 'ProbDict_' + str(n)
  col_name_2 = 'ProbArray_' + str(n)
  col_name_3 = 'NGrams_' + str(n)
  traces_ngrams[col_name_1] = traces_ngrams.apply(lambda row: calc_probabilities(row[col_name_3], all_ngrams_list[n], False), axis = 1)
  traces_ngrams[col_name_2] = traces_ngrams.apply(lambda row: calc_probabilities(row[col_name_3], all_ngrams_list[n], True), axis = 1)

In [16]:
#a few quick sense checks
example_dict = traces_ngrams['ProbDict_2'].iloc[0]
example_array = traces_ngrams['ProbArray_2'].iloc[0]

#check translation functions work
example_dict_converted_to_array = prob_dict_to_array(example_dict)
example_array_converted_to_dict = prob_array_to_dict(example_array, all_ngrams_list[2])

print(np.array_equal(example_dict_converted_to_array, example_array))
print(example_array_converted_to_dict == example_dict)

#check probability array is normalised
print(sum(example_array))

True
True
1.0


In [17]:
#remove columns that are unnecessary for clustering algorithms and distance measure calculations
cols = ['Player', 'GameID', 'AgentName', 'AgentNameOpponent', 'Win', 'FinalScore', 'TotalRounds']
traces_cardcount_slim = traces_cardcount.drop(cols, axis = 1)

In [20]:
#calculate distance and affinity matrices for jenen-shannon and l_k norm distance functions
def symm_distance_matrix(df, distance_func, func_param = False):
    traces = df.tolist()
    index_combinations = list(combinations(range(len(traces)), 2))

    #we branch here so we can use both lk-norm and Jensen-Shannon in this function
    if not func_param:
      distance_values = [distance_func(traces[i],traces[j]) for i, j in index_combinations]
    else:
      distance_values = [distance_func(func_param, traces[i],traces[j]) for i, j in index_combinations]

    num_rows = len(df)
    distance_matrix = pd.DataFrame(index=range(num_rows), columns=range(num_rows))

    for (i, j), distance_value in zip(index_combinations, distance_values):
        distance_matrix.at[i, j] = distance_value
        distance_matrix.at[j, i] = distance_value  # mirror the value

    return distance_matrix.fillna(0)  # fill NaN values with zeros for diagonal elements

#affinity function used when computing fully connected affinity matrix
def connected_affinity_func(x, gamma):
  return np.exp(-gamma * (x**2))

#calculate fully connected affinity matrix
def connected_affinity_matrix(dist_matrix, gamma):
  eps = 0.00000001
  matrix = np.vectorize(connected_affinity_func)(dist_matrix, gamma)
  #note we put a lower bound on the entries in the infinity matrix to make sure
  #we have a fully connected graph
  return np.vectorize(max)(matrix, eps)

#calculate k-nearest neighbour affinity matrix
def knn_affinity_matrix(dist_matrix, k):
  # Find indices of k-nearest neighbors for each data point
  neigh = NearestNeighbors(n_neighbors=k + 1, metric='precomputed').fit(dist_matrix)
  _, indices = neigh.kneighbors()

  # Create knn affinity matrix
  aff_matrix = np.zeros_like(dist_matrix, dtype=float)
  for i in range(dist_matrix.shape[0]):
      aff_matrix[i, indices[i, 1:]] = 1.0  # Assign 1 to the k-nearest neighbors

  # Make the matrix symmetric
  aff_matrix = 0.5 * (aff_matrix + aff_matrix.T)

  return aff_matrix

#function to calculate Jensen-Shannon distance
def kl_divergence(p, q):
  eps = 1e-10
  return np.sum(np.where(p < eps, 0, np.where(q < eps, 0, p * np.log((p + eps) / (1.0 * q + eps)))))

def jensen_shannon_distance(p, q):
  m = 0.5 * (p + q)
  return 0.5 * (kl_divergence(p, m) + kl_divergence(q, m))

#function to compute l_k norm, here p and q are arrays of doubles
def l_k_norm(k, p, q):
  return np.sum(np.abs(p - q) ** k) ** (1/(1.0*k))

#calculate distance, fully connected and k-nearest neighbour affinity matrices for l_k norm for card count playtraces
dist_matrices = {}
connected_affinity_matrices = {}
knn_affinity_matrices = {}
traces_cardcount_arrays = traces_cardcount_slim.apply(lambda row: np.array(row), axis=1)
#pdb.set_trace()
for k in k_norms:
  key_norm = 'CardCount_lknorm_' + str(k)
  connected_affinity_matrices[key_norm] = {}
  knn_affinity_matrices[key_norm] = {}
  dist_matrices[key_norm] = symm_distance_matrix(traces_cardcount_arrays, l_k_norm, k)
  for gamma in np.arange(gamma_min, gamma_max, gamma_stepsize):
    key_gamma =  'gamma_' + str(gamma)
    connected_affinity_matrices[key_norm][key_gamma] = connected_affinity_matrix(dist_matrices[key_norm], gamma)
  for nn in range(nearest_neighbours_min, nearest_neighbours_max, nearest_neighbours_stepsize):
    key_nn =  'knn_' + str(nn)
    knn_affinity_matrices[key_norm][key_nn] = knn_affinity_matrix(dist_matrices[key_norm], nn)

#for dbcan we also need to normalise the playtraces to reduce the space we need to search over
#epsilon to between zero and one. Note that the Jensen-Shannon distance measure is by defintion less than one
#so we only need to do this for our card count playtraces
traces_cardcount_slim_normalised = traces_cardcount_slim.apply(lambda row: row/np.linalg.norm(row), axis = 1)
traces_cardcount_slim_normalised_arrays = traces_cardcount_slim_normalised.apply(lambda row: np.array(row), axis=1)
dist_matrices_normalised = {}
for k in k_norms:
  key_norm = 'CardCount_lknorm_' + str(k)
  dist_matrices_normalised[key_norm] = symm_distance_matrix(traces_cardcount_slim_normalised_arrays, l_k_norm, k)

#calculate distance, fully connected and k-nearest neighbour affinity matrices for N-Gram playtraces
for n in range(ngram_min, ngram_max + 1, ngram_stepsize):
  key_gram = 'N_Gram_' + str(n)
  connected_affinity_matrices[key_gram] = {}
  knn_affinity_matrices[key_gram] = {}
  col_name = 'ProbArray_' + str(n)
  dist_matrices[key_gram] = symm_distance_matrix(traces_ngrams[col_name], jensen_shannon_distance)
  for gamma in np.arange(gamma_min, gamma_max, gamma_stepsize):
    key_gamma =  'gamma_' + str(gamma)
    connected_affinity_matrices[key_gram][key_gamma] = connected_affinity_matrix(dist_matrices[key_gram], gamma)
  for nn in range(nearest_neighbours_min, nearest_neighbours_max, nearest_neighbours_stepsize):
    key_nn =  'knn_' + str(nn)
    knn_affinity_matrices[key_gram][key_nn] = knn_affinity_matrix(dist_matrices[key_gram], nn)

In [21]:
#functions to perfom clustering analysis
def sa_kmedoids(dist_matrix, num_clusters):
  clusterer = KMedoids(n_clusters=num_clusters,
                       metric='precomputed',
                       method='pam',
                       init='k-medoids++',
                       max_iter=300,
                       random_state= 0).fit(dist_matrix)
  #when we use precomputed as the metric, the algorithm returns the indices for the cluster centres only
  return clusterer.inertia_, clusterer.medoid_indices_, clusterer.labels_

def sa_kmeans(data, num_clusters):
  clusterer = KMeans(n_clusters=num_clusters,
                      init='k-means++',
                      n_init= 'warn',
                      max_iter=300,
                      tol=0.0001,
                      verbose=0,
                      random_state=0,
                      copy_x=True,
                      algorithm='lloyd').fit(data)
  return clusterer.inertia_, clusterer.cluster_centers_, clusterer.labels_

def sa_dbscan(dist_matrix, minPts, epsilon):
  dbscan_clustering = DBSCAN(eps= epsilon, min_samples= minPts, metric = 'precomputed').fit(dist_matrix)
  return dbscan_clustering.labels_

#spectral clcustering using pre-computed affinity matrx
def sa_spectral_clustering_AM(affinity_matrix, num_clusters):
  spec_clustering_AM = SpectralClustering(n_clusters= num_clusters,
                                        random_state=0,
                                        affinity = 'precomputed',
                                        assign_labels='kmeans').fit(affinity_matrix)
  return spec_clustering_AM.labels_

In [22]:
#functions to generate plots and output files
def output_silhouette_plot(outputfilename, silhouette_samples, silhouette_avg, cluster_labels):
  n_clusters = len(np.unique(cluster_labels))

  #Create a subplot with 1 row and 1 columns
  fig, ax1 = plt.subplots(1,1, clear = True)
  fig.set_size_inches(7, 3.5)

  # The silhouette coefficient can range from -1, 1
  ax1.set_xlim([-0.1, 1])
  # The (n_clusters+1)*10 is for inserting blank space between silhouette
  # plots of individual clusters, to demarcate them clearly.
  ax1.set_ylim([0, len(silhouette_samples) + (n_clusters + 1) * 10])

  y_lower = 10
  for i in range(n_clusters):
      # Aggregate the silhouette scores for samples belonging to
      # cluster i, and sort them
      ith_cluster_silhouette_values = silhouette_samples[cluster_labels == i]

      ith_cluster_silhouette_values.sort()

      size_cluster_i = ith_cluster_silhouette_values.shape[0]
      y_upper = y_lower + size_cluster_i

      color = cm.nipy_spectral(float(i) / n_clusters)
      ax1.fill_betweenx(
          np.arange(y_lower, y_upper),
          0,
          ith_cluster_silhouette_values,
          facecolor=color,
          edgecolor=color,
          alpha=0.7,
      )

      # Label the silhouette plots with their cluster numbers at the middle
      ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

      # Compute the new y_lower for next plot
      y_lower = y_upper + 10  # 10 for the 0 samples

  #ax1.set_title("The silhouette plot for the various clusters.")
  ax1.set_xlabel("The silhouette coefficient values for K=" + str(n_clusters))
  ax1.set_ylabel("Cluster label")

  # The vertical line for average silhouette score of all the values
  ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

  ax1.set_yticks([])  # Clear the yaxis labels / ticks
  ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

  #plt.suptitle(
  #    "Silhouette analysis for " + cluster_method_str + " clustering",
  #    fontsize=14,
  #    fontweight="bold",
  #)
  plt.savefig(outputfilename +'.png', format = 'png')
  plt.close()

#output list of silhouette averages with a key (e.g. no of clusters)
def output_silhouette_avgs(outputfilename, silhouette_avgs_dict):
  #output silhouette averages to file
  with open(outputfilename + '.csv', 'w', newline='') as csv_file:
    fieldnames = silhouette_avgs_dict.keys()
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    # Write the header
    writer.writeheader()
    # Write the data
    writer.writerow(silhouette_avgs_dict)

  return None

#output single best silhouette average and corresponding parameter values
def output_best_silhouette_average_and_params(outputfilename, best_sil_avg, param_list, paramlabel_list):
  with open(outputfilename + '.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Best Silhouette Average: ', best_sil_avg])
    for index in range(0, len(param_list)):
      writer.writerow([paramlabel_list[index], param_list[index]])

def output_inertia_plot(outputfilename, inertia_vals_dict, scalar):
  #scale the inertia vals, typically the scalar value will be the inertia for clustering on one cluster
  inertia_vals_array = np.array([value for value in inertia_vals_dict.values()])
  inertia_vals_scaled = inertia_vals_array/scalar
  cluster_list = np.array([cluster for cluster in inertia_vals_dict.keys()])
  inertia_vals_scaled = np.insert(inertia_vals_scaled, 0, 1.0)
  cluster_list = np.insert(cluster_list, 0, 1)

  #plot as a line plot
  fig, ax = plt.subplots(num=1,clear=True)
  ax.plot(cluster_list, inertia_vals_scaled)
  ax.set_xticks(cluster_list)
  ax.set_xlabel("Number of clusters")
  ax.set_ylabel("Scaled inertia")
  plt.savefig(outputfilename +'.png', format = 'png')
  plt.close()

#function to plot card count playtraces
def cardcount_playtrace_comparison(outputfilename, trace_list, label_list, card_types, legendOn = True, ylimit = 0):
  #look at evolution of number of cards of each type per round.
  #traces should all be of the same length
  maxRounds =   int(trace_list[0].shape[1]/17)
  noOfCardTypes = len(card_types)
  noOfSubPlotCols = 5
  noOfSubPlotRows = max(2, math.floor(noOfCardTypes/noOfSubPlotCols) + 1)
  fig, axs = plt.subplots(noOfSubPlotRows, noOfSubPlotCols, figsize = (10,10))
  for i in range(0,noOfSubPlotRows):
      for j in range(0,noOfSubPlotCols):
          cardIndex = noOfSubPlotRows*j + i
          if cardIndex >= len(card_types):
              axs[i,j].set_visible(False)
          else:
              card_type = card_types[cardIndex]
              card_col = [card_type + "_R" + str(r) for r in range(0,maxRounds)]
              card_max = 0
              for (index, trace) in enumerate(trace_list):
                  axs[i,j].plot(range(0,maxRounds), trace[card_col].iloc[0], label = label_list[index])
                  tmp_card_max = int(trace[card_col].iloc[0].max())
                  if tmp_card_max > card_max:
                      card_max = tmp_card_max

              #set labels and limits
              axs[i,j].set_title(card_type)
              axs[i,j].set_xlabel('Round')
              if ylimit == 0:
                  axs[i,j].set_ylim((0,card_max+2))
              else:
                  axs[i,j].set_ylim((0,1))
              #axs[i,j].set_ylim((0,card_max))
              axs[i,j].set_xticks(ticks = range(0, maxRounds,10))

          #tighten subplots layout
          fig.tight_layout()

  #add overal legend to figure
  if legendOn:
      axs[0,noOfSubPlotRows - 1].legend(loc = (1.2,-0.8))

  #output to file
  plt.savefig(outputfilename +'.png', format = 'png')
  plt.close()

#plot cluster centroids based on cardcount playtraces, that are outputted directly from sklearn clustering algorithms
def plot_cluster_centroids(outputfilename, cluster_centers, card_types, legendOn = True, ylimit = 0):
  #cluster centres outputted from sklearn are 2D arrays with rows corresponding to clusters and columns corresponding to length of trace
  no_of_clusters = cluster_centres.shape[0]
  maxRounds = int(cluster_centres.shape[1]/17)
  df_cluster_centres = pd.DataFrame(cluster_centers)
  cols = [card_types[i] + "_R" + str(r) for r in range(0, maxRounds)
          for i in range(0, len(card_types))]
  df_cluster_centres.columns = cols

  trace_list = []
  label_list = []
  for n in range(0, no_of_clusters):
      trace_list.append(pd.DataFrame(df_cluster_centres.iloc[n]).transpose())
      label_list.append(str('Cluster ') + str(n) + str(' Centroid'))
  cardcount_playtrace_comparison(outputfilename, trace_list, label_list, card_types, legendOn, ylimit)

#output a selection of cluster metrics. This includes:
#1. Portion of traces in each cluster that have a given agent name
#2. Portion of traces in a cluster that were from the player that moved first
#3. Portfolio of traces in a cluster that won
def ouput_cluster_metrics(outputfilename, traces, cluster_labels):
  tmp_traces = traces.copy()
  tmp_traces['Cluster'] = cluster_labels

  #loop over attibutes to compute distributions
  df_result = pd.DataFrame()
  for att in ['AgentName', 'Player', 'Win']:
    result = tmp_traces.groupby(['Cluster', att]).size().unstack().fillna(0)
    results_percentage = result.div(result.sum(axis=1), axis=0) * 100
    if att == 'AgentName':
      df_result = results_percentage
    else:
      if att == 'Player':
        results_percentage.columns = ['First Player', 'Second Player']
      else:
        if len(results_percentage.columns) == 2:
          results_percentage.columns = ['Win', 'Loss']
        else:
          results_percentage.columns = ['Win', 'Draw', 'Loss']
      df_result = df_result.join(results_percentage)

  #output to file
  df_result.to_csv(outputfilename + '.csv')

#convert a list of ngram tuples into a list of strings, used in plotting function below
def convert_ngram_tuples_to_strings(ngrams_list):
  ngrams_str = []
  for tuple_item in ngrams_list:
      tuple_str = ''
      for index, element in enumerate(tuple_item):
          if index != (len(tuple_item)-1):
              tuple_str += element + '|'
          else:
              tuple_str += element
      ngrams_str.append(tuple_str)
  return ngrams_str

#function to plot N-Gram distributions side by side
def plot_distribution_comparison(outputfilename, prob_dicts, labels, threshold = 0):
  #find a common domain where all probability values are greater than a given threshold
  common_ngrams = []
  for prob_dict in prob_dicts:
    for key, value in prob_dict.items():
      if (value > threshold) and (key not in common_ngrams):
          common_ngrams.append(key)

  #extract probability arrays for these common n-grams
  prob_arrays = []
  for prob_dict in prob_dicts:
    prob_dict_reduced = {key: prob_dict[key] for key in common_ngrams}
    prob_arrays.append(prob_dict_to_array(prob_dict_reduced))

  #next plot probability distributions

  #need to convert common_ngrams into a list of strings as opposed to tuples containing strings
  common_ngrams_str = convert_ngram_tuples_to_strings(common_ngrams)

  #plot discrete probability distributions side by side

  # Set the width of the bars
  bar_width = 0.35

  counter = 0
  x_values_list = []
  for prob_array in prob_arrays:
    # Calculate the x-coordinates for the bars
    if counter == 0:
      x_values = np.arange(len(common_ngrams_str))
    else:
      x_values = x_values_list[counter - 1] + bar_width
    x_values_list.append(x_values)

    plt.bar(x_values, prob_array, width=bar_width, label = labels[counter])
    counter += 1

  plt.xticks(x_values_list[0] + bar_width * len(prob_arrays) / 2, common_ngrams_str)
  plt.xticks(rotation=90)
  plt.ylim(threshold)
  plt.legend()

  #output to file
  plt.savefig(outputfilename +'.png', format = 'png')
  plt.close()

In [23]:
#We switch off interactive mode for matplotlib as plots will be output to file
plt.ioff()

In [ ]:
#perfom K-means clustering. We only do this for card-count playtraces due to restriction on using Euclidean playtraces
print("Perfoming K-means clustering for card count playtraces using euclidean norm...")
sil_avg = {}
inertia = {}
for n_clusters in range(clusters_min, clusters_max + 1, clusters_stepsize):
  inertia[n_clusters], cluster_centres, cluster_labels = sa_kmeans(traces_cardcount_slim, n_clusters)
  sil_avg[n_clusters] = silhouette_score(traces_cardcount_slim, cluster_labels)
  sil_coeffs = silhouette_samples(traces_cardcount_slim, cluster_labels)

  #output silhouette plots
  outputfilename = google_drive_parent_dir + 'Results_KMeans/' + tag_for_dir_and_filenames + '/' + 'silhouette_plot_KMeans_CardCount_N_' + str(n_clusters) + '_' + tag_for_dir_and_filenames
  output_silhouette_plot(outputfilename, sil_coeffs, sil_avg[n_clusters], cluster_labels)

  #output cluster centroids
  outputfilename = google_drive_parent_dir + 'Results_KMeans/' + tag_for_dir_and_filenames + '/' + 'cluster_centroids_KMeans_CardCount_N_' + str(n_clusters) + '_' + tag_for_dir_and_filenames
  plot_cluster_centroids(outputfilename, cluster_centres, card_types)

  #output cluster metrics
  outputfilename = google_drive_parent_dir + 'Results_KMeans/' + tag_for_dir_and_filenames + '/' + 'cluster_metrics_KMeans_CardCount_N_' + str(n_clusters) + '_' + tag_for_dir_and_filenames
  ouput_cluster_metrics(outputfilename, traces_cardcount, cluster_labels)

#output silhouette averages to file
outputfilename = google_drive_parent_dir + 'Results_KMeans/' + tag_for_dir_and_filenames + '/' + 'silhouette_averages_KMeans_CardCount_' + tag_for_dir_and_filenames
output_silhouette_avgs(outputfilename, sil_avg)

#output a scaled inertia value plot
outputfilename = google_drive_parent_dir + 'Results_KMeans/' + tag_for_dir_and_filenames + '/' + 'inertia_plot_KMeans_CardCount_' + tag_for_dir_and_filenames
scalar, _, _ = sa_kmeans(traces_cardcount_slim, 1)
output_inertia_plot(outputfilename, inertia, scalar)


In [ ]:
#perform K-medoids clustering for card count playtraces.
print("Perfoming K-medoids clustering for card count playtraces using l_k-norm...")

for k in k_norms:
  key_norm = 'CardCount_lknorm_' + str(k)
  sil_avg = {}
  inertia = {}
  for n_clusters in range(clusters_min, clusters_max + 1, clusters_stepsize):
    inertia[n_clusters], cluster_indices, cluster_labels = sa_kmedoids(dist_matrices[key_norm], n_clusters)
    sil_avg[n_clusters] = silhouette_score(dist_matrices[key_norm], cluster_labels, metric = 'precomputed')
    sil_coeffs = silhouette_samples(dist_matrices[key_norm], cluster_labels, metric = 'precomputed')

    #output silhouette plots
    outputfilename = google_drive_parent_dir + 'Results_KMedoids/' + tag_for_dir_and_filenames + '/' + 'silhouette_plot_KMedoids_CardCount_N_' + str(n_clusters) + '_k_' + str(k) + '_' + tag_for_dir_and_filenames
    output_silhouette_plot(outputfilename, sil_coeffs, sil_avg[n_clusters], cluster_labels)

    #output cluster centroids, converting indices to playtraces
    cluster_centres = traces_cardcount_slim.iloc[cluster_indices]
    outputfilename = google_drive_parent_dir + 'Results_KMedoids/' + tag_for_dir_and_filenames + '/' + 'cluster_centroids_KMedoids_CardCount_N_' + str(n_clusters) + '_k_' + str(k) + '_' + tag_for_dir_and_filenames
    plot_cluster_centroids(outputfilename, cluster_centres, card_types)

    #output cluster metrics
    outputfilename = google_drive_parent_dir + 'Results_KMedoids/' + tag_for_dir_and_filenames + '/' + 'cluster_metrics_KMedoids_CardCount_N_' + str(n_clusters) + '_k_' + str(k) + '_' + tag_for_dir_and_filenames
    ouput_cluster_metrics(outputfilename, traces_cardcount, cluster_labels)

  #output silhouette averages to file
  outputfilename = google_drive_parent_dir + 'Results_KMedoids/' + tag_for_dir_and_filenames + '/' + 'silhouette_averages_KMedoids_CardCount' + '_k_' + str(k) + '_' + tag_for_dir_and_filenames
  output_silhouette_avgs(outputfilename, sil_avg)

  #output a scaled inertia value plot
  outputfilename = google_drive_parent_dir + 'Results_KMedoids/' + tag_for_dir_and_filenames + '/' + 'inertia_plot_KMedoids_CardCount' + '_k_' + str(k) + '_' + tag_for_dir_and_filenames
  scalar, _, _ = sa_kmedoids(dist_matrices[key_norm], 1)
  output_inertia_plot(outputfilename, inertia, scalar)

In [ ]:
#perform K-medoids clustering for N-Gram playtraces.
print("Perfoming K-medoids clustering for N-Gram playtraces using Jensen-Shannon norm...")

for n in range(ngram_min, ngram_max + 1, ngram_stepsize):
  key_gram = 'N_Gram_' + str(n)
  sil_avg = {}
  inertia = {}
  for n_clusters in range(clusters_min, clusters_max + 1, clusters_stepsize):
    inertia[n_clusters], cluster_indices, cluster_labels = sa_kmedoids(dist_matrices[key_gram], n_clusters)
    sil_avg[n_clusters] = silhouette_score(dist_matrices[key_gram], cluster_labels, metric = 'precomputed')
    sil_coeffs = silhouette_samples(dist_matrices[key_gram], cluster_labels, metric = 'precomputed')

    #output silhouette plots
    outputfilename = google_drive_parent_dir + 'Results_KMedoids/' + tag_for_dir_and_filenames + '/' + 'silhouette_plot_KMedoids_NGram_' + str(n) + '_N_' + str(n_clusters) + '_' + tag_for_dir_and_filenames
    output_silhouette_plot(outputfilename, sil_coeffs, sil_avg[n_clusters], cluster_labels)

    #output cluster centroids (probability distributions in this case)
    col_name = 'ProbDict_' + str(n)
    cluster_centres = traces_ngrams[col_name].iloc[cluster_indices]
    labels = ['Cluster ' + str(n) for n in range(0, n_clusters)]
    outputfilename = google_drive_parent_dir + 'Results_KMedoids/' + tag_for_dir_and_filenames + '/' + 'cluster_centroids_KMedoids_NGram_' + str(n) + '_N_' + str(n_clusters) + '_' + tag_for_dir_and_filenames
    plot_distribution_comparison(outputfilename, cluster_centres, labels, thresholds[n])

  #output silhouette averages to file
  outputfilename = google_drive_parent_dir + 'Results_KMedoids/' + tag_for_dir_and_filenames + '/' + 'silhouette_averages_KMedoids_NGram_' + str(n) + '_' + tag_for_dir_and_filenames
  output_silhouette_avgs(outputfilename, sil_avg)

  #output a scaled inertia value plot
  outputfilename = google_drive_parent_dir + 'Results_KMedoids/' + tag_for_dir_and_filenames + '/' + 'inertia_plot_KMedoids_NGram_' + str(n) + '_' + tag_for_dir_and_filenames
  scalar, _, _ = sa_kmedoids(dist_matrices[key_gram], 1)
  output_inertia_plot(outputfilename, inertia, scalar)


In [ ]:
#perform DBSCAN clustering for card count playtraces using l_k-norm
print("Perfoming DBSCAN clustering for card count playtraces using l_k-norm...")

for k in k_norms:
  key_norm = 'CardCount_lknorm_' + str(k)
  #note that for DBSCAN, inertia is not a valid metric (dependent on spherical clusters), so we look for the
  #best silhouette average and just output this result.
  best_sil_avg = -10000
  best_sil_coeffs = None
  best_minpts = 0
  best_epsilon = 0
  best_cluster_labels = None
  best_noise_ratio = 0 #this is the noise ratio in the case with the highest silhouette average
  for minpts in range(minpts_min, minpts_max, minpts_stepsize):
    for eps in np.arange(epsilon_min, epsilon_max, epsilon_stepsize):
      cluster_labels = sa_dbscan(dist_matrices_normalised[key_norm], minpts, eps)
      #in DBSCAN anything with a label of '-1' is treated as noise
      #so we need to do the following:
      #1. Keep a record of the portion of traces that are classified as noise, too many and the results should be ignored
      #2. Filter our traces to remove traces that are considered as noise, prior to computing the silhouette average
      noise_ratio = np.sum(cluster_labels == -1)/len(traces_cardcount_slim_normalised)
      cluster_labels_no_noise = cluster_labels[cluster_labels > -1]
      indices_to_remove = [i for i, value in enumerate(cluster_labels) if value == -1]
      dist_matrix_no_noise = dist_matrices_normalised[key_norm].drop(index=indices_to_remove, columns=indices_to_remove)
      no_clusters_found = len(np.unique(cluster_labels_no_noise))
      if (no_clusters_found < 2):
        #in this case we cannot compute a silhouette score and we just output the centroids
        #how do we determine the best choice of hyperparameters in this case?
        i = 0 #add necessary code here
      else:
        sil_avg = silhouette_score(dist_matrix_no_noise, cluster_labels_no_noise, metric = 'precomputed')
        if sil_avg > best_sil_avg:
          best_sil_avg = sil_avg
          best_sil_coeffs = silhouette_samples(dist_matrix_no_noise, cluster_labels_no_noise, metric = 'precomputed')
          best_minpts = minpts
          best_epsilon = eps
          best_cluster_labels = cluster_labels_no_noise
          best_noise_ratio = noise_ratio

  #output silhouette plots
  if (best_sil_avg > 0):
    #output silhouette plots
    outputfilename = google_drive_parent_dir + 'Results_DBSCAN/' + tag_for_dir_and_filenames + '/' + 'silhouette_plot_DBSCAN_CardCount_eps_' + str(round(best_epsilon,2)) + '_minpts_' + str(best_minpts) + '_k_' + str(k) + '_' + tag_for_dir_and_filenames
    output_silhouette_plot(outputfilename, best_sil_coeffs, best_sil_avg, best_cluster_labels)

    #output best silhouette average result
    outputfilename = google_drive_parent_dir + 'Results_DBSCAN/' + tag_for_dir_and_filenames + '/' + 'best_silhouette_avg_DBSCAN_CardCount' + '_k_' + str(k) + '_' + tag_for_dir_and_filenames
    params = [best_minpts, best_epsilon, best_noise_ratio]
    paramlabels_list = ['minpts', 'epsilon', 'noise ratio']
    output_best_silhouette_average_and_params(outputfilename, best_sil_avg, params, paramlabels_list)
  else:
    print("No clustering found for DBSCAN for card count playtraces with l_" + str(k) + "-norm")
    #TODO: Anything else we can do here?

  #TODO:Visualisation and cluster centroids


In [ ]:
print("Perfoming DBSCAN clustering for N-Gram playtraces using Jensen-Shannon norm...")

for n in range(ngram_min, ngram_max + 1, ngram_stepsize):
  key_gram = 'N_Gram_' + str(n)
  best_sil_avg = -10000
  best_sil_coeffs = None
  best_minpts = 0
  best_epsilon = 0
  best_cluster_labels = None
  best_noise_ratio = 0 #this is the noise ratio in the case with the highest silhouette average
  for minpts in range(minpts_min, minpts_max, minpts_stepsize):
    for eps in np.arange(epsilon_min, epsilon_max, epsilon_stepsize):
      cluster_labels = sa_dbscan(dist_matrices[key_gram], minpts, eps)
      noise_ratio = np.sum(cluster_labels == -1)/len(traces_ngrams)
      cluster_labels_no_noise = cluster_labels[cluster_labels > -1]
      indices_to_remove = [i for i, value in enumerate(cluster_labels) if value == -1]
      dist_matrix_no_noise = dist_matrices[key_gram].drop(index=indices_to_remove, columns=indices_to_remove)
      no_clusters_found = len(np.unique(cluster_labels_no_noise))
      if (no_clusters_found < 2):
        #in this case we cannot compute a silhouette score and we just output the centroids
        #how do we determine the best choice of hyperparameters in this case?
        i = 0 #add necessary code here
      else:
        sil_avg = silhouette_score(dist_matrix_no_noise, cluster_labels_no_noise, metric = 'precomputed')
        if sil_avg > best_sil_avg:
          best_sil_avg = sil_avg
          best_sil_coeffs = silhouette_samples(dist_matrix_no_noise, cluster_labels_no_noise, metric = 'precomputed')
          best_minpts = minpts
          best_epsilon = eps
          best_cluster_labels = cluster_labels_no_noise
          best_noise_ratio = noise_ratio

  #output silhouette plots
  if (best_sil_avg > 0):
    #output silhouette plots
    outputfilename = google_drive_parent_dir + 'Results_DBSCAN/' + tag_for_dir_and_filenames + '/' + 'silhouette_plot_DBSCAN_NGram_N_' + str(n) + '_eps_' + str(round(best_epsilon,2)) + '_minpts_' + str(best_minpts) + '_' + tag_for_dir_and_filenames
    output_silhouette_plot(outputfilename, best_sil_coeffs, best_sil_avg, best_cluster_labels)

    #output best silhouette average result
    outputfilename = google_drive_parent_dir + 'Results_DBSCAN/' + tag_for_dir_and_filenames + '/' + 'best_silhouette_avg_DBSCAN_NGram_N_' + str(n) + '_' + tag_for_dir_and_filenames
    params = [best_minpts, best_epsilon, best_noise_ratio]
    paramlabels_list = ['minpts', 'epsilon', 'noise ratio']
    output_best_silhouette_average_and_params(outputfilename, best_sil_avg, params, paramlabels_list)
  else:
    print("No clustering found for DBSCAN for N-gram playtraces with N=" + str(n))
    #TODO: Anything else we can do here?

  #TODO:Visualisation and cluster centroids

In [ ]:
print("Performing spectral clustering for card count playtraces using fully connected graphs with l_k-norm...")

for k in k_norms:
  key_norm = 'CardCount_lknorm_' + str(k)
  best_sil_avg = -10000
  best_sil_coeffs = None
  best_gamma = 0
  best_no_clusters = 0
  best_cluster_labels = None
  for no_clusters in range(clusters_min, clusters_max, clusters_stepsize):
    for gamma in np.arange(gamma_min, gamma_max, gamma_stepsize):
      key_gamma =  'gamma_' + str(gamma)
      cluster_labels = sa_spectral_clustering_AM(connected_affinity_matrices[key_norm][key_gamma], no_clusters)
      sil_avg = silhouette_score(dist_matrices[key_norm], cluster_labels, metric = 'precomputed')
      if sil_avg > best_sil_avg:
        best_sil_avg = sil_avg
        best_sil_coeffs = silhouette_samples(dist_matrices[key_norm], cluster_labels, metric = 'precomputed')
        best_gamma = gamma
        best_no_clusters = no_clusters
        best_cluster_labels = cluster_labels

  #output silhouette plots
  if (best_sil_avg > 0):
    #output silhouette plots
    outputfilename = google_drive_parent_dir + 'Results_SPCluster_AM/' + tag_for_dir_and_filenames + '/' + 'silhouette_plot_SPCluster_AM_CardCount_gamma_' + str(round(best_gamma,2)) + '_N_' + str(best_no_clusters) + '_k_' + str(k) + '_' + tag_for_dir_and_filenames
    output_silhouette_plot(outputfilename, best_sil_coeffs, best_sil_avg, best_cluster_labels)

    #output best silhouette average result
    outputfilename = google_drive_parent_dir + 'Results_SPCluster_AM/' + tag_for_dir_and_filenames + '/' + 'best_silhouette_avg_SPCluster_AM_CardCount' + '_k_' + str(k) + '_' + tag_for_dir_and_filenames
    params = [best_no_clusters, best_gamma]
    paramlabels_list = ['Clusters', 'gamma']
    output_best_silhouette_average_and_params(outputfilename, best_sil_avg, params, paramlabels_list)
  else:
    print("No clustering found for Spectral Clustering with fully connected affinity matrix for card count playtraces with l_" + str(k) + "-norm")
    #TODO: Anything else we can do here?

  #TODO:Visualisation and cluster centroids



In [ ]:
print("Performing spectral clustering for N-Gram playtraces using fully connected graphs with Jensen-Shannon distance metric...")

for n in range(ngram_min, ngram_max + 1, ngram_stepsize):
  key_gram = 'N_Gram_' + str(n)
  best_sil_avg = -10000
  best_sil_coeffs = None
  best_gamma = 0
  best_no_clusters = 0
  best_cluster_labels = None
  for no_clusters in range(clusters_min, clusters_max, clusters_stepsize):
    for gamma in np.arange(gamma_min, gamma_max, gamma_stepsize):
      key_gamma =  'gamma_' + str(gamma)
      cluster_labels = sa_spectral_clustering_AM(connected_affinity_matrices[key_gram][key_gamma], no_clusters)
      sil_avg = silhouette_score(dist_matrices[key_gram], cluster_labels, metric = 'precomputed')
      if sil_avg > best_sil_avg:
        best_sil_avg = sil_avg
        best_sil_coeffs = silhouette_samples(dist_matrices[key_gram], cluster_labels, metric = 'precomputed')
        best_gamma = gamma
        best_no_clusters = no_clusters
        best_cluster_labels = cluster_labels

  #output silhouette plots
  if (best_sil_avg > 0):
    #output silhouette plots
    outputfilename = google_drive_parent_dir + 'Results_SPCluster_AM/' + tag_for_dir_and_filenames + '/' + 'silhouette_plot_SPCluster_AM_NGram_' + str(n) +'_gamma_' + str(round(best_gamma,2)) + '_N_' + str(best_no_clusters) + '_' + tag_for_dir_and_filenames
    output_silhouette_plot(outputfilename, best_sil_coeffs, best_sil_avg, best_cluster_labels)

    #output best silhouette average result
    outputfilename = google_drive_parent_dir + 'Results_SPCluster_AM/' + tag_for_dir_and_filenames + '/' + 'best_silhouette_avg_SPCluster_AM_NGram_' + str(n) + '_' + tag_for_dir_and_filenames
    params = [best_no_clusters, best_gamma]
    paramlabels_list = ['Clusters', 'gamma']
    output_best_silhouette_average_and_params(outputfilename, best_sil_avg, params, paramlabels_list)
  else:
    print("No clustering found for Spectral Clustering with fully connected affinity matrix for NGram playtraces with N=" + str(n))
    #TODO: Anything else we can do here?

  #TODO:Visualisation and cluster centroids

In [24]:
print("Performing spectral clustering for card count playtraces using K-Nearest Neighbours affinity matrix with l_k-norm...")

for k in k_norms:
  key_norm = 'CardCount_lknorm_' + str(k)
  best_sil_avg = -10000
  best_sil_coeffs = None
  best_nn = 0
  best_no_clusters = 0
  best_cluster_labels = None
  for no_clusters in range(clusters_min, clusters_max, clusters_stepsize):
    for nn in np.arange(nearest_neighbours_min, nearest_neighbours_max, nearest_neighbours_stepsize):
      key_knn =  'knn_' + str(nn)
      cluster_labels = sa_spectral_clustering_AM(knn_affinity_matrices[key_norm][key_knn], no_clusters)
      sil_avg = silhouette_score(dist_matrices[key_norm], cluster_labels, metric = 'precomputed')
      if sil_avg > best_sil_avg:
        best_sil_avg = sil_avg
        best_sil_coeffs = silhouette_samples(dist_matrices[key_norm], cluster_labels, metric = 'precomputed')
        best_nn = nn
        best_no_clusters = no_clusters
        best_cluster_labels = cluster_labels

  #output silhouette plots
  if (best_sil_avg > 0):
    #output silhouette plots
    outputfilename = google_drive_parent_dir + 'Results_SPCluster_AM/' + tag_for_dir_and_filenames + '/' + 'silhouette_plot_SPCluster_AM_CardCount_kNN_' + str(best_nn) + '_N_' + str(best_no_clusters) + '_k_' + str(k) + '_' + tag_for_dir_and_filenames
    output_silhouette_plot(outputfilename, best_sil_coeffs, best_sil_avg, best_cluster_labels)

    #output best silhouette average result
    outputfilename = google_drive_parent_dir + 'Results_SPCluster_AM/' + tag_for_dir_and_filenames + '/' + 'best_silhouette_avg_SPCluster_AM_CardCount' + '_k_' + str(k) + '_' + tag_for_dir_and_filenames
    params = [best_no_clusters, best_nn]
    paramlabels_list = ['Clusters', 'k-NearestNeighbours']
    output_best_silhouette_average_and_params(outputfilename, best_sil_avg, params, paramlabels_list)
  else:
    print("No clustering found for Spectral Clustering with k_Nearest Neighbours affinity matrix for card count playtraces with l_" + str(k) + "-norm")
    #TODO: Anything else we can do here?

  #TODO:Visualisation and cluster centroids

Performing spectral clustering for card count playtraces using K-Nearest Neighbours affinity matrix with l_k-norm...


/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ma

In [25]:
print("Performing spectral clustering for N-Gram playtraces using K-Nearest Neighbours affinity matrix with Jensen-Shannon...")

for n in range(ngram_min, ngram_max + 1, ngram_stepsize):
  key_gram = 'N_Gram_' + str(n)
  best_sil_avg = -10000
  best_sil_coeffs = None
  best_nn = 0
  best_no_clusters = 0
  best_cluster_labels = None
  for no_clusters in range(clusters_min, clusters_max, clusters_stepsize):
    for nn in np.arange(nearest_neighbours_min, nearest_neighbours_max, nearest_neighbours_stepsize):
      key_knn =  'knn_' + str(nn)
      cluster_labels = sa_spectral_clustering_AM(knn_affinity_matrices[key_gram][key_knn], no_clusters)
      sil_avg = silhouette_score(dist_matrices[key_gram], cluster_labels, metric = 'precomputed')
      if sil_avg > best_sil_avg:
        best_sil_avg = sil_avg
        best_sil_coeffs = silhouette_samples(dist_matrices[key_gram], cluster_labels, metric = 'precomputed')
        best_nn = nn
        best_no_clusters = no_clusters
        best_cluster_labels = cluster_labels

  #output silhouette plots
  if (best_sil_avg > 0):
    #output silhouette plots
    outputfilename = google_drive_parent_dir + 'Results_SPCluster_AM/' + tag_for_dir_and_filenames + '/' + 'silhouette_plot_SPCluster_AM_NGram_' + str(n) + '_kNN_' + str(best_nn) + '_N_' + str(best_no_clusters) + '_' + tag_for_dir_and_filenames
    output_silhouette_plot(outputfilename, best_sil_coeffs, best_sil_avg, best_cluster_labels)

    #output best silhouette average result
    outputfilename = google_drive_parent_dir + 'Results_SPCluster_AM/' + tag_for_dir_and_filenames + '/' + 'best_silhouette_avg_SPCluster_AM_NGram_' + str(n) +  '_' + tag_for_dir_and_filenames
    params = [best_no_clusters, best_nn]
    paramlabels_list = ['Clusters', 'k-NearestNeighbours']
    output_best_silhouette_average_and_params(outputfilename, best_sil_avg, params, paramlabels_list)
  else:
    print("No clustering found for Spectral Clustering with k_Nearest Neighbours affinity matrix for N-Gram playtraces with N=" + str(n))
    #TODO: Anything else we can do here?

  #TODO:Visualisation and cluster centroids

Performing spectral clustering for N-Gram playtraces using K-Nearest Neighbours affinity matrix with Jensen-Shannon...


/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ma